In [65]:
# !pip list

In [66]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta

In [67]:
# Sample Inputs
investment_amount = 10000  # Total investment amount
start_date = "2024-01-01"  # Investment period start date
end_date = "2024-05-01"  # Investment period end date

In [68]:
stocks = [
    {"Ticker": "ADANIPORTS.NS", "Weightage": 0.0082},
    {"Ticker": "APOLLOHOSP.NS", "Weightage": 0.0061},
    {"Ticker": "ASIANPAINT.NS", "Weightage": 0.0195},
    # {"Ticker": "AXISBANK.NS", "Weightage": 0.0257},
    # {"Ticker": "BAJAJFINSV.NS", "Weightage": 0.012},
    # {"Ticker": "BAJFINANCE.NS", "Weightage": 0.0237},
    # {"Ticker": "BHARTIARTL.NS", "Weightage": 0.0233},
    # {"Ticker": "BPCL.NS", "Weightage": 0.0046},
    # {"Ticker": "BRITANNIA.NS", "Weightage": 0.0052},
    # {"Ticker": "CIPLA.NS", "Weightage": 0.0068},
    # {"Ticker": "COALINDIA.NS", "Weightage": 0.0051},
    # {"Ticker": "DIVISLAB.NS", "Weightage": 0.0077},
    # {"Ticker": "DRREDDY.NS", "Weightage": 0.0067},
    # {"Ticker": "EICHERMOT.NS", "Weightage": 0.0049},
    # {"Ticker": "GRASIM.NS", "Weightage": 0.0085},
    # {"Ticker": "HCLTECH.NS", "Weightage": 0.0153},
    # {"Ticker": "HDFCBANK.NS", "Weightage": 0.081},
    # {"Ticker": "HDFCLIFE.NS", "Weightage": 0.0072},
    # {"Ticker": "HEROMOTOCO.NS", "Weightage": 0.0043},
    # {"Ticker": "HINDALCO.NS", "Weightage": 0.0094},
    # {"Ticker": "HINDUNILVR.NS", "Weightage": 0.0267},
    # {"Ticker": "ICICIBANK.NS", "Weightage": 0.069},
    # {"Ticker": "INDUSINDBK.NS", "Weightage": 0.0085},
    # {"Ticker": "INFY.NS", "Weightage": 0.0766},
    # {"Ticker": "ITC.NS", "Weightage": 0.0303},
    # {"Ticker": "JSWSTEEL.NS", "Weightage": 0.0094},
    # {"Ticker": "KOTAKBANK.NS", "Weightage": 0.0351},
    # {"Ticker": "LT.NS", "Weightage": 0.0274},
    # {"Ticker": "MARUTI.NS", "Weightage": 0.0137},
    # {"Ticker": "NESTLEIND.NS", "Weightage": 0.0087},
    # {"Ticker": "NTPC.NS", "Weightage": 0.0099},
    # {"Ticker": "ONGC.NS", "Weightage": 0.0078},
    # {"Ticker": "POWERGRID.NS", "Weightage": 0.0104},
    # {"Ticker": "RELIANCE.NS", "Weightage": 0.1286},
    # {"Ticker": "SBILIFE.NS", "Weightage": 0.0065},
    # {"Ticker": "SBIN.NS", "Weightage": 0.0254},
    # {"Ticker": "SHREECEM.NS", "Weightage": 0.0046},
    # {"Ticker": "SUNPHARMA.NS", "Weightage": 0.0134},
    # {"Ticker": "TATACONSUM.NS", "Weightage": 0.0066},
    # {"Ticker": "TATAMOTORS.NS", "Weightage": 0.0105},
    # {"Ticker": "TATASTEEL.NS", "Weightage": 0.0137},
    # {"Ticker": "TCS.NS", "Weightage": 0.0491},
    # {"Ticker": "TECHM.NS", "Weightage": 0.0105},
    # {"Ticker": "TITAN.NS", "Weightage": 0.0137},
    # {"Ticker": "ULTRACEMCO.NS", "Weightage": 0.0102},
    # {"Ticker": "UPL.NS", "Weightage": 0.006},
    # {"Ticker": "WIPRO.NS", "Weightage": 0.0101},
]

In [69]:
start_date = pd.Timestamp(start_date)
end_date = pd.Timestamp(end_date)
results = []

# Process each stock
for stock in stocks:
    ticker = stock["Ticker"]
    weightage = stock["Weightage"]

    try:
        # Fetch stock data
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty:
            print(f"Data for {ticker} is not available. Skipping.")
            continue

        # Calculate RSI
        data["RSI"] = ta.rsi(data["Adj Close"], length=14)

        # Adjust weightage dynamically
        data["Adjusted Weightage"] = weightage
        data.loc[data["RSI"] > 90, "Adjusted Weightage"] -= 0.003
        data.loc[
            (data["RSI"] > 80) & (data["RSI"] <= 90), "Adjusted Weightage"
        ] -= 0.002
        data.loc[
            (data["RSI"] > 70) & (data["RSI"] <= 80), "Adjusted Weightage"
        ] -= 0.001
        data.loc[data["RSI"] < 20, "Adjusted Weightage"] += 0.003
        data.loc[
            (data["RSI"] >= 20) & (data["RSI"] < 30), "Adjusted Weightage"
        ] += 0.002
        data.loc[
            (data["RSI"] >= 30) & (data["RSI"] <= 70), "Adjusted Weightage"
        ] += 0.001

        # Calculate investment per stock
        investment_per_stock = investment_amount * weightage

        # Calculate shares
        data["Shares"] = investment_per_stock / data["Adj Close"]
        data["Adjusted Shares"] = (
            investment_amount * data["Adjusted Weightage"]
        ) / data["Adj Close"]
        
		

        # Filter for start_date and end_date
        filtered_data = data.loc[(data.index == start_date) | (data.index == end_date)]
        if filtered_data.empty:
            print(
                f"No data for {ticker} on {start_date.date()} or {end_date.date()}. Skipping."
            )
            continue

        # Append results for each filtered row
        for index, row in filtered_data.iterrows():
            try:
                results.append(
                    {
                        "Ticker": ticker,
                        "Date": index.date(),
                        "Weightage": weightage,
                        "Adjusted Weightage": (
                            row["Adjusted Weightage"]
                            if pd.notna(row["Adjusted Weightage"])
                            else None
                        ),
                        "Open": row["Open"] if pd.notna(row["Open"]) else None,
                        "Close": (
                            row["Adj Close"] if pd.notna(row["Adj Close"]) else None
                        ),
                        "RSI": row["RSI"] if pd.notna(row["RSI"]) else None,
                        "Shares": row["Shares"] if pd.notna(row["Shares"]) else None,
                        "Adjusted Shares": (
                            row["Adjusted Shares"]
                            if pd.notna(row["Adjusted Shares"])
                            else None
                        ),
                    }
                )
            except KeyError as ke:
                print(f"KeyError while processing row for {ticker}: {ke}")
            except Exception as e:
                print(f"Error processing row for {ticker}: {e}")

    except ZeroDivisionError as zde:
        print(f"ZeroDivisionError for {ticker}: {zde}")
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Convert results to DataFrame
summary = pd.DataFrame(results)

# Debugging: Check the actual column names in summary
print("Column names in summary:", summary.columns)

# Reorder columns to match the output format
expected_columns = [
    "Ticker",
    "Date",
    "Weightage",
    "Adjusted Weightage",
    "Open",
    "Close",
    "RSI",
    "Shares",
    "Adjusted Shares",
]
missing_columns = [col for col in expected_columns if col not in summary.columns]

if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    # Reorder columns if all are present
    summary = summary[expected_columns]

# Display the summary
print("\nInvestment Summary:")
print(summary)

# Save to CSV for reference
summary.to_csv("investment_summary.csv", index=False)
print("\nSummary saved to 'investment_summary.csv'")

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Error processing row for ADANIPORTS.NS: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing row for APOLLOHOSP.NS: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error processing row for ASIANPAINT.NS: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Column names in summary: RangeIndex(start=0, stop=0, step=1)
Missing columns: ['Ticker', 'Date', 'Weightage', 'Adjusted Weightage', 'Open', 'Close', 'RSI', 'Shares', 'Adjusted Shares']

Investment Summary:
Empty DataFrame
Columns: []
Index: []

Summary saved to 'investment_summary.csv'
